# Basic Usage

Illustrate the basic use of agents

## Packages and options

In [1]:
# Add parent folder to Python path
import sys
sys.path.insert(0, "..")

from typing import Any

import numpy as np
from pydantic import Field

from cosi_consumer_framework import (
    Environment, 
    Agent, 
    AgentPerception, 
    ChoiceSet
)


## Building a simple model

### Environment

The environment is the central source of truth. It holds all objective information
as well all agents and assets.

In our simple example the environment is only characterized by a temperature
that is randomly set in each year. To achieve this, we manipulate the step 
method by setting the temperature before asking all agents to act which is done 
by the parent's step method.

Note that the Environment (like Agents, Assets, and all the other classes) is a 
Pydantic Model and we, therefore, can describe the  fields and also check incoming 
data.

In [2]:
class TemperatureEnvironment(Environment):
    temperature: float = Field(15.0, description="Current temperature in degrees Celsius")

    def step(self):
        self.temperature = np.random.randint(0,40)
        print(f"{self.year}: Environment temperature changed to: {self.temperature}")
        super().step()

### Agents

Agents perceive information from the environment. The perceived information is 
then used to trigger a ChoiceSet based on the given information which is then
used actually do a choice.

In the following we first define the perception and the ChoiceSet. Afterwards we
implement the Agent itself.

#### Perception

Perceiving information is a two part process. First, information is extracted from
the environment. The result of this extraction implemented through the method
*get_information_from_environment* is the undistorted information. This information
is used to instantiate the perception.

In the second step, this objective truth is distorted using the *distort_information* method.
Both of these method must be implemented for the *AgentPerception* object. 

The *AgentPerception* class has already implemented a *perceive* method that 
uses the agent as well as environment to call the two perception methods.

In [3]:
class TemperaturePerception(AgentPerception):
    temperature: float = Field(..., description="Perceived temperature in degrees Celsius")
    
    @classmethod
    def get_information_from_environment(
            cls, agent: Any, environment: TemperatureEnvironment
        ) -> dict:
        """Extract information from the environment.
        
        Args:
            agent: The agent that is perceiving the information.
            environment: The environment in which the agent is located.

        Returns:
            Dictionary with the relevant information
        """
        return {"temperature": environment.temperature}
    
    def distort_information(self, agent: Any) -> None:
        """Distort the perceived information.
        
        Args:
            agent: The agent that is perceiving the information.
        """
        self.temperature += agent.temperature_bias


#### Choice Set

The choice sets is created using the *trigger* function that takes the agent
and the perception as input. Within the trigger we extract all information needed
to do the evaluation of each option. The *evaluate* method then evaluates each choice 
option. We therefore must implement both of these methods.

In [4]:
class DrinkChoice(ChoiceSet):
    options: list[str] = Field(..., description="Available drink options")
    scores: dict[str, float] = Field(
        default_factory=dict, 
        description="Scores for each drink option"
    )
    temperature: float = Field(..., description="Perceived temperature in degrees Celsius")

    agent: Any = Field(..., description="The agent making the choice")

    @classmethod
    def trigger(cls, agent: Any, perception: TemperaturePerception) -> "DrinkChoice":
        """Trigger the choice set based on the agent's perception.
        
        Args:
            agent: The agent making the choice.
            perception: The agent's perception of the environment.

        Returns:
            An instance of the DrinkChoice class.
        """
        # Note: In a real scenario, options could be dynamically generated based 
        # on perception and stored drinks in the environment.
        options = ["hot chocolate", "iced tea"]
        return cls( 
            options=options,
            agent=agent,
            temperature=perception.temperature
        )
    
    def evaluate(self):
        """Create evaluation score for each drink option"""
        self.scores = {
            o: (
                self.agent.drink_preference[o] 
                + self.agent.temperature_adjustment[o] * self.temperature
            ) 
            for o in self.options
        } 

DrinkChoice(options=["hot chocolate", "iced tea"], agent=None, temperature=20)

DrinkChoice(options=['hot chocolate', 'iced tea'], scores={}, temperature=20.0, agent=None)

### Agent

Finally we can specify the agent with all of its attributes:

In [5]:
class Person(Agent):
    drink_preference: dict[str, float] = Field(
        ...,
        description="Score for each drink option at zero degree Celsius"
    )
    temperature_adjustment: dict[str, float] = Field(
        ..., 
        description="Adjustment factor for drink preference based on temperature"
    )
    temperature_bias: float = Field(
        0.0, description="Bias in temperature perception (e.g., due to clothing)"
    )

    def perceive(self, environment: Environment) -> TemperaturePerception:
        return TemperaturePerception.perceive(self, environment)
    
    def trigger_choice(self, perception: TemperaturePerception):
        return DrinkChoice.trigger(agent=self, perception=perception)
    
    def choose(self, options: Any, perception: Any):
        best_option = max(options.scores, key=options.scores.get)
        print(
            f"{self.id}: It is {perception.temperature} of felt temperature!"
            f"I drink: {best_option}"
        )

## Simulation

### Model setup

We create the environment and register one agent:

In [6]:
my_env = TemperatureEnvironment()
me = Person(
    id="Alice",
    drink_preference={"hot chocolate": 20.0, "iced tea": 0},
    temperature_adjustment={"hot chocolate": -1, "iced tea": 1},
    temperature_bias=2.0
)
my_env.add(me)

In [7]:
for i in range(20):
    print(f"\n--- Simulation step {i+1} ---")
    my_env.step()


--- Simulation step 1 ---
2020: Environment temperature changed to: 38
Person.Alice: It is 40.0 of felt temperature!I drink: iced tea

--- Simulation step 2 ---
2021: Environment temperature changed to: 6
Person.Alice: It is 8.0 of felt temperature!I drink: hot chocolate

--- Simulation step 3 ---
2022: Environment temperature changed to: 3
Person.Alice: It is 5.0 of felt temperature!I drink: hot chocolate

--- Simulation step 4 ---
2023: Environment temperature changed to: 22
Person.Alice: It is 24.0 of felt temperature!I drink: iced tea

--- Simulation step 5 ---
2024: Environment temperature changed to: 34
Person.Alice: It is 36.0 of felt temperature!I drink: iced tea

--- Simulation step 6 ---
2025: Environment temperature changed to: 3
Person.Alice: It is 5.0 of felt temperature!I drink: hot chocolate

--- Simulation step 7 ---
2026: Environment temperature changed to: 25
Person.Alice: It is 27.0 of felt temperature!I drink: iced tea

--- Simulation step 8 ---
2027: Environment t